In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import tkinter as tk
from tkinter import filedialog
import os
import random
import shutil

# Create a GUI window
root = tk.Tk()
root.withdraw()

# Open a file dialog for selecting a folder
folder_path = filedialog.askdirectory()

# Print the path of the selected folder
print("Selected folder:", folder_path)


# Set the path to the folder of images
path = folder_path

# Set the percentage of images to use for testing
test_percent = float(input("Enter percentage of images to use for testing (0-1): "))

# Set the paths for the training and testing folders
train_path = os.path.join(path, "train")
test_path = os.path.join(path, "test")

# Create the training and testing folders if they don't exist
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

# Get a list of all the image files in the folder
image_files = [f for f in os.listdir(path) if f.endswith(".jpg") or f.endswith(".png")]

# Calculate the number of images to use for testing
num_test = int(test_percent * len(image_files))

# Shuffle the list of image files
random.shuffle(image_files)

# Move the first num_test images to the testing folder
for i in range(num_test):
    src = os.path.join(path, image_files[i])
    dst = os.path.join(test_path, image_files[i])
    shutil.move(src, dst)

# Move the remaining images to the training folder
for i in range(num_test, len(image_files)):
    src = os.path.join(path, image_files[i])
    dst = os.path.join(train_path, image_files[i])
    shutil.move(src, dst)

print("Done.")


In [2]:
df = pd.read_csv('train_data.csv')

In [3]:
print(df)

       label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0          1       0       0       0       0       0       0       0       0   
1          0       0       0       0       0       0       0       0       0   
2          1       0       0       0       0       0       0       0       0   
3          4       0       0       0       0       0       0       0       0   
4          0       0       0       0       0       0       0       0       0   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
41995      0       0       0       0       0       0       0       0       0   
41996      1       0       0       0       0       0       0       0       0   
41997      7       0       0       0       0       0       0       0       0   
41998      6       0       0       0       0       0       0       0       0   
41999      9       0       0       0       0       0       0       0       0   

       pixel8  ...  pixel774  pixel775 

In [4]:
def sampling(tdata,val):
    len1= int(len(tdata)*val)
    t1 = tdata.sample(frac=1)
    val_data = t1.iloc[0:len1,:]
    train = t1.iloc[len1:,:]
    return val_data,train

v,d = sampling(df,0.2)

In [5]:
print(v.shape)

(8400, 785)


In [6]:
def generator(train,batch_size):
    count = 0
    while count<len(train):
        data = train.iloc[count:count+batch_size,:]
        X = data.iloc[:,1:]
        Y = data.iloc[:,0]
        yield X.values,Y.values
        count+=batch_size
    
    
gen= generator(d,8)


In [12]:
def generator_new(train,batch_size):
    count = 0
    while count<len(train):
        data = train.iloc[count:count+batch_size,:]
        X = data.iloc[:,1:]
        Y = data.iloc[:,0]
        return X,Y
        count+=batch_size
    
    
gen= generator(d,8)

In [7]:
print(next(gen))

(array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64), array([7, 4, 1, 0, 0, 8, 5, 9], dtype=int64))


In [18]:
class neuralNetwork:
    def __init__(self,learning_rate):
        self.weights = np.array([np.random.randn(),np.random.randn()])
        self.bias = np.random.randn()
        self.learning_rate = learning_rate

    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def sigmoid_deriv(self,x):
        return self.sigmoid(x)*(1-self.sigmoid(x))
    
    def predict(self,input):
        l1 = np.dot(input,self.weights) + self.bias
        l2 = self.sigmoid(l1)
        prediction = l2
        return prediction
    
    def gradients(self,input,target):
        l1 = np.dot(input,self.weights) + self.bias
        l2 = self.sigmoid(l1)
        prediction = l2

        error_prediction = 2*(prediction-target)
        prediction_l1 = self.sigmoid_deriv(l1)
        l1_bias = 1
        l1_weight = (0*self.weights)+(1*input)
        error_bias = (error_prediction*prediction_l1*l1_bias)
        error_weights = (error_prediction*prediction_l1*l1_weight)

        return error_bias,error_weights

    def update_params(self,error_bias,error_weights):
        self.bias = self.bias - (error_bias*self.learning_rate)
        self.weights = self.weights - (error_weights*self.learning_rate)


    def train(self,input,target,iterations):
        errors = []
        for current in range(iterations):
            random_index = np.random.randint(len(input))

            input = input[random_index]
            target = target[random_index]

            #Compute gradients and update weights
            error_bias,error_weights = self.gradients(input,target)
            self.update_params(error_bias,error_weights)

            #Measure error for iterations 
            if current%100==0:
                cum_error = 0
                # Loop through all instances to measure the error
                for each_instance in range(len(input)):
                    data_point = input[each_instance]
                    target = target[each_instance]

                    prediction = self.predict(data_point)
                    error = np.square(prediction-target)
                    cum_error = cum_error + error
                errors.append(cum_error)

        return errors

In [20]:
model_net = []
for i in range(5):
    v,d = sampling(df,0.2)
    v_data = v[:,1:]
    v_label = v[:,0]
    generator1 = generator_new(d,64)
    x,y = generator1
    learning_rate = 0.1
    nn = neuralNetwork(learning_rate)
    nn.train(x,y,50)
    model_net.append(nn)

    
    
    

InvalidIndexError: (slice(None, None, None), slice(1, None, None))

In [ ]:
for model in model_net:
    prediction = model.predict(v_data)
    label_op.append(prediction)
    validation 
    

In [ ]:
label_op = []
input 
for model in model_net:
    prediction = model.predict(v_data)
    label_op.append(prediction)
max = 0
res = label_op[0]
for i in label_op:
    freq = label_op.count(i)
    if freq > max:
        max = freq
        res = i
        
print(res)